In [1]:
import os
from google.colab import drive
drive.mount("drive/")

Mounted at drive/


In [2]:
os.chdir("/content/drive/MyDrive/HETIC PFA")

In [7]:
!ls

Models	Raw_Data  removed_bg_data


In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
  # intersection over union
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [15]:
from tensorflow.keras.utils import CustomObjectScope
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("Models/DeepLabv3+.h5")


In [ ]:
tf.keras.utils.plot_model(model, "DeepLabv3+_model.png")

In [25]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

""" Global parameters """
H = 512
W = 512

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("removed_bg_data")
create_dir("removed_bg_data/AD")
""" Loading model: DeepLabV3+ """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}): 
  model = tf.keras.models.load_model("Models/DeepLabv3+.h5")

# model.summary()

""" Load the dataset """
data_x = glob("/content/drive/MyDrive/HETIC PFA/Raw_Data/AD/*")

for path in tqdm(data_x, total=len(data_x)):
  """ Extracting name """
  name = path.split("/")[-1].split(".")[0]

  """ Read the image """
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  h, w, _ = image.shape
  x = cv2.resize(image, (W, H))
  x = x/255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=0)

  """ Prediction """
  y = model.predict(x)[0]
  y = cv2.resize(y, (w, h))
  y = np.expand_dims(y, axis=-1)
  y = y > 0.5

  photo_mask = y
  background_mask = np.abs(1-y)

  # cv2.imwrite(f"remove_bg/{name}.png", photo_mask*255)
  # cv2.imwrite(f"remove_bg/{name}.png", background_mask*255)

  # cv2.imwrite(f"remove_bg/{name}.png", image * photo_mask)
  # cv2.imwrite(f"remove_bg/{name}.png", image * background_mask)

  masked_photo = image * photo_mask
  background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1)
  background_mask = background_mask * [0, 0, 1]
  final_photo = masked_photo + background_mask
  cv2.imwrite(f"removed_bg_data/AD/{name}.jpg", image * photo_mask)

100%|██████████| 128/128 [04:35<00:00,  2.16s/it]


In [16]:
!pwd

/content/drive/MyDrive/HETIC PFA


In [26]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope

""" Global parameters """
H = 512
W = 512

""" Creating a directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("removed_bg_data")
create_dir("removed_bg_data/NAD")

""" Loading model: DeepLabV3+ """
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}): 
  model = tf.keras.models.load_model("Models/DeepLabv3+.h5")

# model.summary()

""" Load the dataset """
data_x = glob("/content/drive/MyDrive/HETIC PFA/Raw_Data/NAD/*")

for path in tqdm(data_x, total=len(data_x)):
  """ Extracting name """
  name = path.split("/")[-1].split(".")[0]

  """ Read the image """
  image = cv2.imread(path, cv2.IMREAD_COLOR)
  h, w, _ = image.shape
  x = cv2.resize(image, (W, H))
  x = x/255.0
  x = x.astype(np.float32)
  x = np.expand_dims(x, axis=0)

  """ Prediction """
  y = model.predict(x)[0]
  y = cv2.resize(y, (w, h))
  y = np.expand_dims(y, axis=-1)
  y = y > 0.5

  photo_mask = y
  background_mask = np.abs(1-y)

  # cv2.imwrite(f"remove_bg/{name}.png", photo_mask*255)
  # cv2.imwrite(f"remove_bg/{name}.png", background_mask*255)

  # cv2.imwrite(f"remove_bg/{name}.png", image * photo_mask)
  # cv2.imwrite(f"remove_bg/{name}.png", image * background_mask)

  masked_photo = image * photo_mask
  background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1)
  background_mask = background_mask * [0, 0, 1]
  final_photo = masked_photo + background_mask
  cv2.imwrite(f"removed_bg_data/NAD/{name}.jpg", image * photo_mask)

100%|██████████| 126/126 [04:20<00:00,  2.07s/it]


In [ ]:
data_x

In [22]:
ls removed_bg_data/

AD/
